In [43]:
import re
import pandas as pd

def parse_dopamine_menu(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = []
    category, subcategory = None, None
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        # Detect category
        if line.startswith("## "):
            category = line[3:].strip()
            subcategory = None
            i += 1
            continue

        # Detect subcategory
        elif line.startswith("### "):
            subcategory = line[4:].strip()
            i += 1
            continue

        # Detect activity name and details
        elif re.match(r"^\d+\. \*\*.*\*\*$", line):
            name_match = re.match(r"\d+\. \*\*(.*?)\*\*", line)
            name = name_match.group(1).strip() if name_match else None
            details = {}
            i += 1

            # Process details for the activity
            while i < len(lines):
                next_line = lines[i].strip()

                if next_line.startswith("- "):
                    key_value_match = re.match(r"-\s*(.+?):\s*(.+)", next_line)
                    if key_value_match:
                        key = key_value_match.group(1).strip()
                        value = key_value_match.group(2).strip()
                        details[key] = value
                elif re.match(r"^\d+\. \*\*.*\*\*$", next_line):
                    i -= 1  # Step back one line
                    break
                elif not next_line:  # Empty line
                    break
                i += 1

            # Ensure all expected detail keys exist
            expected_keys = ["Energy", "Executive Function", "Spoons", "Cost", "Time", "Boost", "Tags", "Notes"]
            for key in expected_keys:
                if key not in details:
                    details[key] = None

            data.append({
                "Category": category,
                "Subcategory": subcategory,
                "Name": name,
                **details,
            })
        
        i += 1

    return pd.DataFrame(data)

In [44]:
# Parse the file and save it to a table
file_path = 'C:/Users/keith/Downloads/Dopamine_Menu_temp.txt'
df = parse_dopamine_menu(file_path)

In [45]:
display(df)

,Category,Subcategory,Name,Energy,Executive Function,Spoons,Cost,Time,Boost,Tags,Notes
0,Morning Momentum Builders,Gentle Start Options (Low Energy/Executive Fun...,Cozy Cat Cuddles,🔋,🧠,🥄,💰,⏱️,🎯🎯🎯,🌅 🏠 📱 🧘‍♀️,Can be combined with phone scrolling if needed...
1,Morning Momentum Builders,Gentle Start Options (Low Energy/Executive Fun...,Aurora Light Show Wake-Up,🔋,🧠,🥄,💰 (once purchased),⏱️,🎯🎯,🌅 🏠 📱 🧘‍♀️,Set up light show night before; can combine wi...
2,Morning Momentum Builders,Gentle Start Options (Low Energy/Executive Fun...,Quick Favorite Tea Ritual,🔋🔋,🧠,🥄,💰,⏱️,🎯🎯,🌅 🏠 📱 🧘‍♀️,Keep special mug and tea easily accessible
3,Morning Momentum Builders,Active Start Options (Medium Energy),Pinterest Morning Inspiration (15 min),🔋🔋,🧠🧠,🥄,💰,⏱️,🎯🎯🎯,🌅 🏠 🎨,"Focus on favorite categories: home decor, fash..."
4,Morning Momentum Builders,Active Start Options (Medium Energy),Quick Home Refresh,🔋🔋,🧠🧠,🥄🥄,💰,⏱️⏱️,🎯🎯🎯,🌅 🏠 🔄,Pick one small area; combine with favorite mus...
...,...,...,...,...,...,...,...,...,...,...,...
249,Color + Movement,Color-Based Activities,Light Pattern Dance,🔋,🧠,🥄,💰,⏱️,🎯🎯🎯,🏠 🔥,Move through aurora light patterns
250,Mindfulness + Achievement,Mindful Completion,Present-Focus Organization,🔋,🧠,🥄,💰,⏱️,🎯🎯🎯,🏠 🧘‍♀️,Mindfully organize one small area
251,Mindfulness + Achievement,Mindful Completion,Sensory Glass Care,🔋,🧠,🥄,💰,⏱️,🎯🎯🎯,🏠 🧘‍♀️,Mindful attention while cleaning glass
252,Social + Nature,Natural Connection,Shared Nature Observation,🔋,🧠,🥄,💰,⏱️,🎯🎯🎯🎯,🤝 🌞,Watch nature with partner/pets


In [46]:
print(df.size)

2794


In [47]:
# Save the dataframe to a CSV file
df.to_csv('Dopamine_Menu_Table.csv', index=False)

print("The data has been parsed and saved as 'Dopamine_Menu_Table.csv'.")

The data has been parsed and saved as 'Dopamine_Menu_Table.csv'.


In [48]:
df.to_excel('C:/Users/keith/Downloads/Dopamine_Menu_Table.xlsx', index=False)

print("The data has been parsed and saved as 'Dopamine_Menu_Table.xlsx'.")

The data has been parsed and saved as 'Dopamine_Menu_Table.xlsx'.


In [49]:
import pandas as pd
import numpy as np

def process_dopamine_menu(input_file, output_file):
    # Read the Excel file
    df = pd.read_excel(input_file)
    
    # Define emoji mappings
    emoji_mappings = {
        '🔋': 'Minimal - Can do while exhausted',
        '🔋🔋': 'Light - Some energy needed',
        '🔋🔋🔋': 'Moderate - Average energy required',
        '🔋🔋🔋🔋': 'Substantial - Need to feel relatively energetic',
        '🔋🔋🔋🔋🔋': 'High - Need to be at peak energy',
        
        '🧠': 'Minimal - Almost automatic',
        '🧠🧠': 'Light - Simple decision-making',
        '🧠🧠🧠': 'Moderate - Some planning needed',
        '🧠🧠🧠🧠': 'Complex - Multiple steps/decisions',
        '🧠🧠🧠🧠🧠': 'Heavy - Significant planning/organization',
        
        '🥄': 'Minimal recovery needed',
        '🥄🥄': 'Short recovery (15-30 mins)',
        '🥄🥄🥄': 'Medium recovery (1-2 hours)',
        '🥄🥄🥄🥄': 'Long recovery (2-4 hours)',
        '🥄🥄🥄🥄🥄': 'Major recovery needed (rest of day)',
        
        '💰': 'Free',
        '💰💰': 'Under $10',
        '💰💰💰': '$10-30',
        '💰💰💰💰': '$30-100',
        '💰💰💰💰💰': 'Over $100',
        
        '⏱️': 'Quick (5-15 mins)',
        '⏱️⏱️': 'Short (15-30 mins)',
        '⏱️⏱️⏱️': 'Medium (30-60 mins)',
        '⏱️⏱️⏱️⏱️': 'Long (1-2 hours)',
        '⏱️⏱️⏱️⏱️⏱️': 'Extended (2+ hours)',
        
        '🎯': 'Mild mood lift',
        '🎯🎯': 'Notable improvement',
        '🎯🎯🎯': 'Significant boost',
        '🎯🎯🎯🎯': 'Major mood elevation',
        '🎯🎯🎯🎯🎯': 'Peak happiness'
    }

    # Tag mappings
    tag_mappings = {
        '🌅': 'Morning-friendly',
        '🌆': 'Afternoon slump fighter',
        '🎄': 'Holiday season special',
        '🎂': 'Birthday period boost',
        '💝': 'Self-care focused',
        '🤝': 'Social connection',
        '🏠': 'Home-based',
        '🌞': 'Weather dependent',
        '🔄': 'Can be combined with tasks',
        '⚡': 'Quick win',
        '🎨': 'Creative',
        '📱': 'Screen-free',
        '🧘‍♀️': 'Calming',
        '🔥': 'Energizing'
    }

    # Function to replace emojis in a single value
    def replace_emojis(value):
        if pd.isna(value):
            return value
        if not isinstance(value, str):
            return value
        
        # First try direct replacement
        if value in emoji_mappings:
            return emoji_mappings[value]
        
        # For Tags column, split and replace individual tags
        if ' ' in value:  # Assumes tags are space-separated
            tags = value.split(' ')
            translated = [tag_mappings.get(tag, tag) for tag in tags]
            return ', '.join(filter(None, translated))
        
        return value

    # Process each column
    for column in df.columns:
        df[column] = df[column].apply(replace_emojis)

    # Save to new Excel file
    df.to_excel(output_file, index=False)
    print(f"Processed file saved as: {output_file}")

# Usage
input_file = 'C:/Users/keith/Downloads/Dopamine_Menu_Table.xlsx'
output_file = 'C:/Users/keith/Downloads/Dopamine_Menu_Table_Translated.xlsx'
process_dopamine_menu(input_file, output_file)

Processed file saved as: C:/Users/keith/Downloads/Dopamine_Menu_Table_Translated.xlsx
